In [13]:
import pymysql
import requests
import bs4
#첫번째 미션 
# company info
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
res = requests.get(url)
html_str = res.text
bs_obj = bs4.BeautifulSoup(html_str)
data_head = bs_obj.find_all('th')
data = bs_obj.find_all('td')

columns = [i.text for i in data_head]

page_data = []
temp = []
target = ['\r','\n','\t']
for i in data:
    if len(temp) == len(columns):
        page_data.append(temp)
        temp = []
    if len(temp) == 7:
        x = i.text
        for j in target:
            x = x.replace(j,'')
        temp.append(x)
    else:
        temp.append(i.text)


# sql code
conn = pymysql.connect(host='localhost',user='root',password='1111',
                       db='INVESTAR',charset='utf8')
cur = conn.cursor()
# sql part
cur.execute(f'create table if not exists company_info'
            f'({columns[0]} char(25),'
            f'{columns[1]} char(6))')

sql="""insert into company_info values (%s, %s);"""
for i,data in enumerate(page_data):
    
    sql = """insert into company_info (회사명,종목코드) values (%s, %s);"""   
    cur.execute(sql, (data[0], data[1]))

OperationalError: (1054, "Unknown column '회사명' in 'field list'")

In [7]:
from keras.models import Sequential,load_model
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf

#seed 값 설정 
seed= 0
np.random.seed(seed)
tf.random.set_seed(3)

# 데이터 불러오고 데이터 입력 
df = pd.read_csv('./data/sonar.csv', header=None)

dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

#문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y=e.transform(Y_obj)

# 학습 셋 과 테스트 셋 구분
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)


#모델 설정
model = Sequential()
model.add(Dense(24, input_dim = 60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='mean_Squared_error', optimizer='adam',metrics=['accuracy'])

# 모델 실행
model.fit(X_train, Y_train, epochs=130, batch_size=5)

#결과 출력 
print("\n Accuracy: %.4f"%(model.evaluate(X_test,Y_test)[1]))

Epoch 1/130


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "C:\ProgramData\Anaconda3\lib\site

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

dt_clf = DecisionTreeClassifier(random_state=156)
skfold = StratifiedKFold(n_split=3)
n_iter=0
cv_accuracy=[]

for train_index, test_index in skfold.split(features,label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter +=1
    accuracy = np.round(accuracy_score(y_test.pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습데이터 크기 : {2}, 검증 데이터 크기:{3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 :{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy))


    
    
    

TypeError: __init__() got an unexpected keyword argument 'n_split'

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import tensorflow as tf

seed =0
np.random.seed(seed)
tf.random.set_seed(3)

df=pd.read_csv('./data/sonar.csv', header=None)

dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]


e=LabelEncoder()
e.fit(Y_obj)
Y= e.transform(Y_obj)

#10개의 파일로 쪼깸
n_fold=10
skf=StratifiedKFold(n_split=n_fold, shuffle=True, random_state=seed)
#빈 배열
accuracy =[]

#모델의 설정, 컴파일, 실행
for train, test in skf.split(X,Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                 optimizer='adma',
                 mertrics = ['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f"%(model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)
    
#결과 출력
print("\n%.f fold accuracy:"% n_fold, accuracy)


TypeError: __init__() got an unexpected keyword argument 'n_split'